# 🏥 MedGemma 1.5 DICOM Report Generator
**By Dr. João Abrantes** · [GitHub Repo](https://github.com/JAbrantesRadiology/MedGemma1.5ReportGenerator)

Auto-generate structured radiology reports from DICOM images using [Google's MedGemma 1.5 4B](https://huggingface.co/google/medgemma-1.5-4b-it) vision-language model.

---

## ✅ Prerequisites (do these BEFORE clicking Run All)

| Step | What to do | Link |
|------|-----------|------|
| **1. GPU Runtime** | `Runtime → Change runtime type → T4 GPU` | — |
| **2. HuggingFace Account** | Create a free account if you don't have one | [huggingface.co/join](https://huggingface.co/join) |
| **3. Accept MedGemma License** | Visit the model page and click **"Agree and access repository"** | [google/medgemma-1.5-4b-it](https://huggingface.co/google/medgemma-1.5-4b-it) |
| **4. Create Access Token** | Settings → Access Tokens → New token (read permission is enough) | [HF Tokens](https://huggingface.co/settings/tokens) |
| **5. Add Token to Colab** | Click the 🔑 key icon in the left sidebar → Add secret named `HF_TOKEN` | — |

Once all prerequisites are met, click **Runtime → Run all** and wait ~5 min for the model to download.

A **public Gradio link** will appear at the bottom — click it to open the app.

> 💡 **No DICOM files?** Cell 6 below downloads a sample chest X-ray so you can test immediately.

## 1️⃣ GPU Check

In [ ]:
# Verify GPU is available and show VRAM
import subprocess, sys

try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,memory.total,memory.free,driver_version', '--format=csv,noheader'],
        capture_output=True, text=True, check=True
    )
    gpu_info = result.stdout.strip()
    parts = [p.strip() for p in gpu_info.split(',')]
    print(f'✅ GPU detected: {parts[0]}')
    print(f'   Total VRAM:  {parts[1]}')
    print(f'   Free VRAM:   {parts[2]}')
    print(f'   Driver:      {parts[3]}')
    if 'T4' in parts[0]:
        print('\n🎯 T4 GPU — perfect for MedGemma 4B (needs ~9 GB VRAM)')
    elif any(x in parts[0] for x in ['A100', 'V100', 'L4']):
        print(f'\n🚀 {parts[0]} detected — more than enough!')
    else:
        print(f'\n⚠️  Unrecognized GPU. MedGemma 4B needs ~9 GB VRAM.')
except FileNotFoundError:
    print('❌ No GPU detected!')
    print('   Go to: Runtime → Change runtime type → T4 GPU')
    print('   Then re-run this cell.')
    sys.exit(1)

## 2️⃣ Install Dependencies

In [ ]:
%%capture
# Install pinned dependencies for reproducibility
!pip install -q \
    gradio==5.23.3 \
    transformers>=4.52.0 \
    accelerate==1.4.0 \
    pydicom==2.4.4 \
    Pillow>=10.0.0 \
    scipy>=1.11.0 \
    python-dotenv>=1.0.0 \
    huggingface_hub>=0.25.0 \
    bitsandbytes>=0.43.0

print('✅ Dependencies installed')

## 3️⃣ HuggingFace Login

In [ ]:
# Login to HuggingFace using Colab Secrets (preferred) or interactive prompt
import os

try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    if not hf_token:
        raise ValueError('HF_TOKEN is empty')
    os.environ['HF_TOKEN'] = hf_token
    # Verify the token works
    from huggingface_hub import HfApi
    api = HfApi(token=hf_token)
    user_info = api.whoami()
    print(f'✅ Logged in as: {user_info["name"]}')
except Exception as e:
    if 'userdata' in str(type(e).__module__) or 'SecretNotFoundError' in str(type(e).__name__):
        print('⚠️  HF_TOKEN not found in Colab Secrets.')
        print('   Click the 🔑 icon in the left sidebar → New secret → Name: HF_TOKEN')
        print('   Paste your token from: https://huggingface.co/settings/tokens')
        print()
    print('Falling back to interactive login...')
    from huggingface_hub import login
    login()
    print('✅ Logged in via interactive prompt')

## 4️⃣ Clone Repository

In [ ]:
import os

REPO_DIR = 'MedGemma1.5ReportGenerator'

if os.path.exists(REPO_DIR):
    print(f'📂 {REPO_DIR}/ already exists — pulling latest changes...')
    !cd {REPO_DIR} && git pull --ff-only
else:
    print('📥 Cloning repository...')
    !git clone https://github.com/JAbrantesRadiology/MedGemma1.5ReportGenerator.git

%cd {REPO_DIR}
print(f'\n✅ Working directory: {os.getcwd()}')

## 5️⃣ Download Sample DICOM (Optional)

No DICOM files handy? This cell creates a sample chest X-ray DICOM packed into a ZIP file you can upload to the app for testing.

The sample is a **synthetic DICOM** generated with pydicom — it contains a procedurally generated chest X-ray-like image with proper DICOM metadata.

In [ ]:
# Create a synthetic chest X-ray DICOM for testing
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import ExplicitVRLittleEndian, generate_uid
import numpy as np
import tempfile, zipfile, os, datetime

def create_synthetic_cxr_dicom(output_path):
    """Create a synthetic chest X-ray DICOM with realistic metadata."""
    file_meta = pydicom.Dataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.1.1'  # Digital X-Ray
    file_meta.MediaStorageSOPInstanceUID = generate_uid()
    file_meta.TransferSyntaxUID = ExplicitVRLittleEndian

    ds = FileDataset(output_path, {}, file_meta=file_meta, preamble=b'\x00' * 128)

    # Patient info
    ds.PatientName = 'TEST^PATIENT'
    ds.PatientID = 'SAMPLE001'
    ds.PatientBirthDate = '19700101'
    ds.PatientSex = 'O'

    # Study info
    ds.StudyDate = datetime.date.today().strftime('%Y%m%d')
    ds.StudyTime = '120000'
    ds.StudyDescription = 'PA CHEST X-RAY'
    ds.Modality = 'DX'
    ds.BodyPartExamined = 'CHEST'
    ds.ViewPosition = 'PA'
    ds.StudyInstanceUID = generate_uid()
    ds.SeriesInstanceUID = generate_uid()
    ds.SOPInstanceUID = file_meta.MediaStorageSOPInstanceUID
    ds.SOPClassUID = file_meta.MediaStorageSOPClassUID
    ds.SeriesNumber = 1
    ds.InstanceNumber = 1
    ds.InstitutionName = 'MedGemma Test Lab'

    # Image parameters
    rows, cols = 512, 512
    ds.Rows = rows
    ds.Columns = cols
    ds.BitsAllocated = 16
    ds.BitsStored = 12
    ds.HighBit = 11
    ds.PixelRepresentation = 0
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = 'MONOCHROME2'
    ds.RescaleIntercept = 0
    ds.RescaleSlope = 1
    ds.WindowCenter = 2048
    ds.WindowWidth = 4096

    # Generate synthetic chest-like image
    np.random.seed(42)
    img = np.zeros((rows, cols), dtype=np.uint16)
    # Background (air - dark)
    img[:] = 200
    # Body outline (ellipse)
    y, x = np.ogrid[:rows, :cols]
    cy, cx = rows // 2, cols // 2
    body_mask = ((x - cx) / 180) ** 2 + ((y - cy) / 220) ** 2 <= 1
    img[body_mask] = 1500
    # Lung fields (darker ellipses)
    for lx_off in [-70, 70]:
        lung = ((x - (cx + lx_off)) / 80) ** 2 + ((y - (cy - 20)) / 150) ** 2 <= 1
        img[lung] = 600
    # Mediastinum (bright center)
    mediastinum = ((x - cx) / 40) ** 2 + ((y - (cy - 10)) / 130) ** 2 <= 1
    img[mediastinum] = 2200
    # Heart silhouette
    heart = ((x - (cx - 20)) / 65) ** 2 + ((y - (cy + 40)) / 70) ** 2 <= 1
    img[heart] = 2400
    # Add noise for realism
    noise = np.random.normal(0, 50, (rows, cols)).astype(np.int16)
    img = np.clip(img.astype(np.int32) + noise, 0, 4095).astype(np.uint16)

    ds.PixelData = img.tobytes()
    ds.save_as(output_path)
    return output_path

# Create DICOM and package as ZIP
sample_dir = '/content/sample_dicom'
os.makedirs(sample_dir, exist_ok=True)

dcm_path = os.path.join(sample_dir, 'chest_xray_sample.dcm')
create_synthetic_cxr_dicom(dcm_path)

zip_path = '/content/sample_chest_xray.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.write(dcm_path, 'chest_xray_sample.dcm')

size_kb = os.path.getsize(zip_path) / 1024
print(f'✅ Sample DICOM created: {zip_path} ({size_kb:.0f} KB)')
print(f'   Modality: DX (Digital X-Ray) — PA Chest')
print(f'   Image: 512×512 px, 12-bit grayscale')
print(f'\n📎 Upload this ZIP file to the Gradio app to test the pipeline.')

## 6️⃣ Launch the App 🚀

This cell starts the Gradio web app with a **public link** you can open in any browser.

First launch downloads the MedGemma model (~8 GB) — this takes **2-5 minutes** depending on network speed.

In [ ]:
# Patch app.py to enable share=True for public URL
with open("app.py", "r") as f:
    code = f.read()
if "share=False" in code:
    code = code.replace("share=False", "share=True")
    with open("app.py", "w") as f:
        f.write(code)

print("🚀 Starting MedGemma... (model download takes 2-3 min on first run)")
print("📎 A public Gradio link will appear below when ready")
print()

!python app.py